# Understanding and Implementing the MLP-Mixer model from the paper - "MLP-Mixer: An all-MLP Architecture for Vision"

CNNs have been the State Of The Art models for most of the Computer Vision applications till date and with the advent for transformer model in Natural Language Processing, Vision Transformers (ViT) have been the new trend. The latter makes use of the attention concept for prediction. But with MLP-Mixers, the need for complex Neural Networks and attention mechanism is negated. Here, we understand and implement the MLP Mixer from the original paper - https://arxiv.org/pdf/2105.01601.pdf

### Original Abstract

Convolutional Neural Networks (CNNs) are the go-to model for computer vision. Recently, attention-based networks, such as the Vision Transformer, have also become popular. In this paper we show that while convolutions and attention are
both sufficient for good performance, neither of them are necessary. We present MLP-Mixer, an architecture based exclusively on multi-layer perceptrons (MLPs). MLP-Mixer contains two types of layers: one with MLPs applied independently to image patches (i.e. “mixing” the per-location features), and one with MLPs applied across patches (i.e. “mixing” spatial information). When trained on large datasets, or with modern regularization schemes, MLP-Mixer attains competitive scores on image classification benchmarks, with pre-training and inference cost comparable to state-of-the-art models. We hope that these results spark further research beyond the realms of well established CNNs and Transformers.

In [2]:
import torch
import torchvision
import torch.nn as nn

The MLP Mixer model has four main components - Patch embeddings, the Mixer Layer, the MLP Layer, and a classifier. The architecture has two MLP layers within each Mixer layer and comprises of a total of N such Mixer layers. The first MLP layer in each Mixer Layer is the Token-mixing MLP which allows communication between the spatial components. Mathematically, the input table is transposed before sending it as an input for this MLP layer. The second MLP layer is the Channel-mixing MLP which allows communication between different channels within each patch/token. 

<img src="images/Fig1.png" width=1000 height=400/>

Each MLP block has a fully connected layer and a GELU activation function. The patches are Normalized at each layer to avoid the problem of exploding/vanishing gradients. Finally the network ends with a Global Average Pooling layer and classifies based on the softmax function. The network also has smaller components like the Skip Connections and the dropout layer.

Since, most computer vision architectures have components which that mix features
 - in the same spatial locations.
 - across different spatial locations.
 
For the former one, conventionally an N x N convolution layer with pooling characeristics solves the problem easily and using a 1x1 convolution layer, both the above mechanisms can be conducted but with larger kernels. The latest vision models like the ViT model and other self-attention models can perform both the actions, while Mixers can perform (ii) well. Here in this architecture, we use two separate MLP components two perform (i) (token-mixing) and (ii) (channel-mixing). 

After the brief description of the network architecture, we move to discuss the **Patch Embedding** process.

### Patch Embedding

Assuming the input image has dimensions (H,W) and the desired resolution of the patch being (P,P), the number of patches the input image can be divided into is given as 

$$S = HW/P^{2}$$

The Mixer takes a sequence of S non-overlapping patches/tokens which are projected into a *hidden* dimension C. As mixer has two MLP components, with the first one being the Token Mixing MLP, the input to this MLP layer is the columns of input X, that is transpose of X. For the second MLP, the channel-Mixing MLP, the input is the rows of X, that is X itself. As shown in the fig above each MLP has two fully connected layers with non-linearity. 

### Parameter Tying

Another important aspect discussed in the paper was the concept of parameter tying which means that the wights could be shared with the layers. The parameters within the channel-Mixing MLP were tied together to avoid the network growing faster when the hidden dimension C was changed. Since most CNNs involved separable convolutions, with mixers, parameter tying was quite plausible and a more efficient strategy as this lead to sufficient memory saving.  

## Building the MLP-Mixer Model

In [ ]:
# Since the entire network has three main components - 
# 1. Patch Embeddings
# 2. Mixer Layer
# 3. MLP 
# Final network - 

# MLP block
class MLPblock(nn.Module):
    def __init__(self, dim_hidden, mlp_out):
        super(MLPblock, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(dim_hidden, mlp_out), # fully connected layer
            nn.GELU(), # non-linearity
            nn.Linear(mlp_out, dim_hidden) # fully connected layer
        )
    def forward(self, x):
        return self.mlp(x)

## Mixer Layer
class MixerLayer(nn.Module):
    def __init__(self, patches_num, dim_hidden, dim_tokens, dim_channels):
        super(MixerLayer, self).__init__()
        self.token_ln = nn.LinearNorm(dim_hidden)
        self.token_mix = MLPBlock(dim_hidden, dim_tokens)
        self.channel_ln = nn.LinearNorm(dim_hidden)
        self.channel_mix = MLPBlock(dim_hidden, dim_channels)
        
    def forward(self,x):
        out = self.token_ln(x)
        out = out.transpose(1,2)
        out = self.token_mix(out)
        out = out.transpose(1,2)
        out = self.channel_ln(x+out)
        out = self.channel_mix(out)
        
        return x + out
    
class MLP_Mixer(nn.Module):
    def __init__(self, num_classes, num_blocks, size_patch, 
